In [ ]:
!pip install accelerate

In [ ]:
!pip install transformers

In [ ]:
!pip install huggingface-hub

In [3]:
!huggingface-hub cli login

/bin/bash: line 1: huggingface-hub: command not found


In [ ]:
!pip install autoawq

In [ ]:
!pip3 install https://github.com/casper-hansen/AutoAWQ/releases/download/v0.1.6/autoawq-0.1.6+cu118-cp310-cp310-linux_x86_64.whl


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

model_name_or_path = "TheBloke/Synatra-7B-v0.3-RP-AWQ"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    low_cpu_mem_usage=True,
    device_map="cuda:0"
)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
model.generation_config

GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 32000,
  "pad_token_id": 0
}

In [ ]:
prompt = "Tell me about AI"
prompt_template=f'''Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{prompt}

### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
# output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
# print(tokenizer.decode(output[0]))

In [7]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig
translation_generation_config = GenerationConfig(
temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512
)

# Tip: add `push_to_hub=True` to push to the Hub
translation_generation_config.save_pretrained("/tmp", "translation_generation_config.json")

# You could then use the named generation config file to parameterize generation

In [8]:
generation_config = GenerationConfig.from_pretrained("/tmp", "translation_generation_config.json")

In [9]:
output = model.generate(inputs=input_ids,generation_config=generation_config)
print(tokenizer.decode(output[0]))

<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Tell me about AI

### Response:
 AI stands for Artificial Intelligence. It is a type of computer software that can think and learn on its own. It uses algorithms to process information and make decisions. AI can perform tasks that normally require human intelligence, such as recognizing images, understanding natural language, and even making predictions. It is used in many different applications, including robotics, healthcare, finance, and entertainment. Some examples of AI include Siri, Alexa, and Google Assistant. AI has the potential to revolutionize many industries and improve our lives in countless ways.<|im_end|>


### Evaluation

In [12]:
sample = "An abstract composition where swirls of cosmic dust and stars form into the shape of a majestic phoenix, set against the deep blues and purples of space, symbolizing rebirth and the endless possibilities of the universe."
prompt = f'You are an artistic prompt writer of images , create image prompt  like the attached sample {sample}\n'
prompt_template=f'''Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{prompt}

### Response:
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids,generation_config=generation_config)
print(tokenizer.decode(output[0]))




*** Generate:
<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
You are an artistic prompt writer of images , create image prompt  like the attached sample An abstract composition where swirls of cosmic dust and stars form into the shape of a majestic phoenix, set against the deep blues and purples of space, symbolizing rebirth and the endless possibilities of the universe.


### Response:
 "The Cosmic Phoenix"

In the boundless expanse of the universe, a majestic phoenix emerges from the swirling clouds of cosmic dust and stars. Its wings outstretched, wingspan reaching towards infinity, it soars effortlessly through the vast emptiness, its golden feathers shimmering with the light of distant galaxies. Its eyes blaze with a fierce intensity, as if it's searching for something lost or forgotten. Its beak is open wide, ready to sing a song of rebirth and renewal. The phoenix's head tilts upward, its crest of

In [13]:
def print_result(prompt):
  prompt_template=f'''Below is an instruction that describes a task. Write a response that appropriately completes the request.

  ### Instruction:
  {prompt}

  ### Response:
  '''

  print("\n\n*** Generate:")

  input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
  output = model.generate(inputs=input_ids,generation_config=generation_config)
  print(tokenizer.decode(output[0]))




In [14]:
prompt = f'You are a business critic, give me 5 competitor analysis of my business plan of selling AI generated images \n'
print_result(prompt)



*** Generate:
<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.

  ### Instruction:
  You are a business critic, give me 5 competitor analysis of my business plan of selling AI generated images 


  ### Response:
  1. DALL-E: DALL-E is a powerful AI image generator that can create realistic images from text descriptions. It offers a variety of styles and resolutions, making it a strong competitor in the AI image generation market.

2. Midjourney: Midjourney is another AI image generator that specializes in creating detailed and high-quality images from text input. It offers customizable settings for image generation, allowing users to fine-tune the output to their specific needs.

3. StableDiffusion: StableDiffusion is an open-source AI model that can generate realistic images from text prompts. It uses a diffusion-based approach to produce high-quality images with minimal noise and artifacts.

4. OpenAI: OpenAI is a non-pro

#### Experiments with DataFrame

In [15]:
import pandas  as pd

df = pd.DataFrame()
df['price'] = [1,2,3,4,5,6,7]
df['SnP'] = [500,1300,1200,600,800,200,100]
df.head()

,price,SnP
0,1,500
1,2,1300
2,3,1200
3,4,600
4,5,800


In [16]:
price = 4
prompt = f'''Take the atatched dataframe {df} , And check what is value of Snp when price is {price}. Dont return anything else. \n

'''

print_result(prompt)



*** Generate:
<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.

  ### Instruction:
  Take the atatched dataframe    price   SnP
0      1   500
1      2  1300
2      3  1200
3      4   600
4      5   800
5      6   200
6      7   100 , And check what is value of Snp when price is 4. Dont return anything else. 




  ### Response:
  2.5<|im_end|>


In [19]:
price = 5
prompt = f'''Take the attched dataframe \n {df} , And check what is value of Snp when price is {price}. Dont return anything else. \n

'''

print_result(prompt)



*** Generate:
<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.

  ### Instruction:
  Take the attched dataframe 
    price   SnP
0      1   500
1      2  1300
2      3  1200
3      4   600
4      5   800
5      6   200
6      7   100 , And check what is value of Snp when price is 5. Dont return anything else. 




  ### Response:
  100<|im_end|>



#### Experiments to write in Poetic Style

In [20]:
prompt = f'You are a the famous writer Jim Rohn, give me 5 ideas to improve my wealth \n'
print_result(prompt)



*** Generate:
<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.

  ### Instruction:
  You are a the famous writer Jim Rohn, give me 5 ideas to improve my wealth 


  ### Response:
  1. Develop a strong work ethic: Success in wealth-building requires consistent effort and discipline. Set clear goals, prioritize tasks, and strive for continuous improvement.

2. Learn from successful people: Surround yourself with individuals who have achieved financial success and learn from their experiences and strategies. Seek out mentorship and guidance.

3. Build multiple streams of income: Diversify your revenue streams by investing in different assets, businesses, or sources of income. This reduces risk and provides stability during economic downturns.

4. Manage your expenses carefully: Create a budget and track your spending habits closely. Cut unnecessary expenses and redirect funds towards savings and investments.

5. Invest in your

#### Codeing Questions

In [21]:

prompt = f''' Write me python code how to add two numbers\n'''
print_result(prompt)



*** Generate:
<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.

  ### Instruction:
   Write me python code how to add two numbers


  ### Response:
   To add two numbers in Python, you can use the built-in "+" operator. Here's a simple example:

```python
# Define two numbers
num1 = 5
num2 = 10

# Add the numbers
result = num1 + num2

# Print the result
print("Result:", result)
```

This code will output "Result: 15". You can replace "num1" and "num2" with any other numbers you want to add together.<|im_end|>


In [22]:

prompt = f''' Write code to compare embeddings of 5 items and show which are nearest or close in meaning \n
'''
print_result(prompt)



*** Generate:
<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.

  ### Instruction:
   Write code to compare embeddings of 5 items and show which are nearest or close in meaning 



  ### Response:
   To compare the embeddings of 5 items and show which are nearest or close in meaning, we can use the cosine similarity function. The cosine similarity measures the cosine of the angle between two vectors, which ranges from -1 to 1. A value of 1 indicates that the two vectors are aligned and have similar meanings, while a value of -1 indicates they have opposite meanings. Here's a step-by-step guide on how to do this in Python using the scikit-learn library:

1. Import the necessary libraries:
```python
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import pairwise_distances
```

2. Load the embedding data into a NumPy array:
```python
embs = np.load('embeddings.npy')
```

3. Calcula

#### Multiple Results Evaluation

In [24]:
sample = "An abstract composition where swirls of cosmic dust and stars form into the shape of a majestic phoenix, set against the deep blues and purples of space, symbolizing rebirth and the endless possibilities of the universe."
prompt = f'You are an artistic prompt writer of images , create 2 image prompt  like the attached sample {sample}\n'
print_result(prompt)



*** Generate:
<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.

  ### Instruction:
  You are an artistic prompt writer of images , create 2 image prompt  like the attached sample An abstract composition where swirls of cosmic dust and stars form into the shape of a majestic phoenix, set against the deep blues and purples of space, symbolizing rebirth and the endless possibilities of the universe.


  ### Response:
  1. An ethereal phoenix rising from the ashes of a supernova, its fiery wings spreading out like celestial tendrils reaching towards the stars, illuminating the dark expanse of space with its radiant glow.

2. A serene phoenix perched gracefully atop a nebula cloud, its feathers softly stirring the cosmic gas around it, creating intricate patterns of light and shadow that dance and twirl like cosmic fireflies.<|im_end|>
